## 🔮 mī lyte: RAG architectures + config

_WIP - NOT FOR DISTRIBUTION_

_Backend notebook: handles retrieval-augemented generation (RAG) initialization, (local) model selection via Ollama, fine-tuning, configuration, and knowledge base curation, for_ 🍂 _mī lyte System 1: an evidence-based informal mindfulness skills + intrapersonal resilience asset recommendation engine._

> `mi_lyte_system01_config.ipynb`<br>
> Simone J. Skeen x Claude Code (02-02-2026)

### 1. Prepare
Installs, imports, requisite packages; customizes outputs.
***
> **Dependencies:** i.) Install via `%pip install -r requirements.txt` from project root before running; ii.) Pull embedding model (terminal) via `ollama pull nomic-embed-text`.

In [ ]:
%%capture

%pip install -r ../requirements.txt

In [ ]:
import numpy as np, os, pandas as pd, streamlit as st, warnings

from dotenv import load_dotenv
from pathlib import Path

import langchain, pydantic
print("LangChain:", langchain.__version__)
print("Pydantic:", pydantic.__version__)

from langchain_community.document_loaders import PyMuPDFLoader
#from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import Ollama
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

pd.options.mode.copy_on_write = True

pd.set_option(
    'display.max_columns',
    None,
    )

pd.set_option(
    'display.max_rows',
    None,
    )

for c in (FutureWarning, UserWarning):
    warnings.simplefilter(
        action = 'ignore',
        category = c,
        )

# load environment variables from project root .env

load_dotenv(Path('..') / '.env')

### 2. Import
Imports standalone `dialogue_stream.py` > `query_and_stream()` function.
***

In [ ]:
# set wd to src/ (notebook's own directory)

CODE_DIR = Path().resolve()
os.chdir(CODE_DIR)
%pwd

In [ ]:
from dialogue_stream import query_and_stream # type: ignore
from config import SYSTEM_PROMPT, LLM_PARAMS, EMBEDDING_MODEL, RETRIEVER_PARAMS, PROMPT_TEMPLATE # type: ignore

### 3. Preprocess + Initialize
Loads, splits, chunks knowledge base documents; initializes LLM, retriever; vectorizes.
***

#### 3a. Load, inspect

In [ ]:
# knowledge base directory (loaded from .env)

KNOW_DIR = os.path.expanduser(os.environ['KNOW_DIR'])

# load pdf knowledge base

pdf_paths = [
    os.path.join(KNOW_DIR, "mbqr_manual_rag_db.pdf"),
    os.path.join(KNOW_DIR, "mbqr_scripts_rag_db.pdf"),
    os.path.join(KNOW_DIR, "poems_of protest_resistance_empowerment_rag_db_prelim.pdf"),
    ]

all_documents = []

# group pdf by file

for path in pdf_paths:
#    loader = PyPDFLoader(path)
    loader = PyMuPDFLoader(path)
    docs = loader.load()
    all_documents.append(docs) ### append "list of lists"

    print(f"{os.path.basename(path)}: {len(docs)} p/p. loaded")
    
# spot check: leading 500 characters, each file

for i, doc_pages in enumerate(all_documents):
    print(f"\n--------------- knowledge source {i+1} ---------------")
    print(f"\n")
    print(doc_pages[0].page_content[:1000])

#### 3b. Vectorize

In [ ]:
# flatten into single list

flat_documents = [page for doc in all_documents for page in doc]

# chunk into "chunked" excerpts

splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap = 200, 
    separators = ['\n\n', '\n', ' ', ''],
    )

chunked_documents = splitter.split_documents(flat_documents)
print(f"chunks chunked: {len(chunked_documents)}")

# embed + create FAISS vector db

embedding = OllamaEmbeddings(model = EMBEDDING_MODEL)
db = FAISS.from_documents(chunked_documents, embedding)

# save

db.save_local("faiss_index")

In [ ]:
# load locally saved vector db

#db = FAISS.load_local(
#    "faiss_index", 
#    embeddings = embedding,
#    allow_dangerous_deserialization = True,
#    )

#### 3c. `system_prompt` + `PromptTemplate()` 💭
Imported from `config.py`.

In [ ]:
print(SYSTEM_PROMPT)

#### 3d. Input 💬 `query`

In [ ]:
query = '''
    I sometimes struggle with negative feelings toward my body
    '''

In [ ]:
#     I have too much to deal with today! I feel so overwhelmed I can't even start
#     I sometimes struggle with negative feelings toward my body
#     Please tell me an inspiring quote. The world feels like too much lately

#### 3e. Configure LLM, `PromptTemplate()`

Anaconda Prompt (anaconda3) > `ollama pull deepseek-r1:14b`

##### Default (`chain_type = 'stuff'`)

In [ ]:
# config llm via ollama

llm = Ollama(**LLM_PARAMS)

# config prompttemplate (default)

prompt = PROMPT_TEMPLATE

# set up retriever

retriever = db.as_retriever(**RETRIEVER_PARAMS)

        ### SJS 8/7: alternate search_type options below...custom fx tktk...

#retriever = db.as_retriever(
#    search_type = 'mmr',
#    search_kwargs = {
#        'k': 10, 
#        'fetch_k': 20, 
#        'lambda_mult': 0.5,
#        }
#    )

#retriever = db.as_retriever(
#    search_type = 'similarity_score_threshold',
#    search_kwargs = {'score_threshold': 0.8},
#    )

In [ ]:
# non-streaming qa_chain config

#qa_chain = RetrievalQA.from_chain_type(
#    llm = llm,
#    retriever = retriever,
#    chain_type = 'stuff', 
#    return_source_documents = True,
#    chain_type_kwargs = {'prompt': prompt},
#    )

# query

#query = query
#result = qa_chain({"query": query})

#print("Answer:", result['result'])

#for doc in result['source_documents']:
#    print("\nSource:", doc.metadata)
#    print(doc.page_content[:500])  # Truncated content

### 4. Query
Prompt QA chain, inspect / rate response.
***

In [ ]:
query_and_stream(
    llm, 
    retriever, 
    query,
    prompt_template = prompt,
    show_sources = True,
    )

> End of mi_lyte_system01_config.ipynb